<div class="alert alert-block alert-info">
<b>Sentence BERT를 이용한 내용 기반 국문 저널 추천 시스템</b>
</div>

# I. 데이터 준비 및 전처리

---
# Setting

In [1]:
# setting path
import sys
sys.path.append('..') # parent directory 경로 추가

from common import *
from my import *

%matplotlib inline
%config InlineBackend.figure_format='retina'

# random seed 고정 
import os, random
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
#     tf.random.set_seed(seed) # Tensorflow 사용시 
SEED = 777
set_seeds(SEED)

# Fonts found
Hiragino Maru Gothic Pro /System/Library/Fonts/ヒラギノ丸ゴ ProN W4.ttc
Noto Sans Gothic /System/Library/Fonts/Supplemental/NotoSansGothic-Regular.ttf
Apple SD Gothic Neo /System/Library/Fonts/AppleSDGothicNeo.ttc
AppleGothic /System/Library/Fonts/Supplemental/AppleGothic.ttf
# Setting Korean
=> AppleGothic set

# Compledted importing and setting


In [3]:
## KoNLPY 및 Mecab 설치
# Homebrew 설치 필요

# !pip install konlpy
# !pip install mecab-python
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [2]:
import os
import pickle
from tqdm import tqdm

import joblib
from joblib import Parallel, delayed, parallel_backend

import collections
from heapq import nlargest
from operator import itemgetter

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt, Mecab

import math
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

---
# 데이터 읽기

사전 완료된 전처리:
- 논문 10개 미만 저널은 제거 됨 (기존 600여개 -> 268개 저널로 감소)
- 학술대회 논문집 제거 됨

추가로 수행할 전처리:
- 결측치 제거
- 텍스트(제목, 키워드) 전처리

In [17]:
df = pd.read_csv('./data/df_complete_filtered.csv', index_col=0, dtype='string')
df.head(3)

,title,abstract,keywords,journal,type
JAKO200622234005936,AHP 기법을 활용한 CRM 평가요소의 상대적 중요도 분석,고객관계관리(CRM)의 전략적 중요성이 증대함에 따라 CRM에 대한 성과평가가 최근...,성과평가;고객관계관리;,CRM연구,JA
JAKO200622234005941,인터넷 오픈마켓 거래안전 요인과 소비자신뢰의 관계 연구,본 연구는 우리나라 전자상거래 업계에서 새로운 업태로 급격하게 성장하고 있는 인터넷...,인터넷오픈마켓;소비자신뢰;소비자만족;거래위험;,CRM연구,JA
JAKO200622234005947,시스템 다이내믹스를 활용한 정보 기술 수용에 대한 동태적 모형 개발 - 휴대 전화 ...,기존의 초기 기술 수용 모델(TAM)과 후기 기술 수용 모델(PAM)을 사용한 연구...,시스템 다이내믹스;휴대 전화;,CRM연구,JA


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103070 entries, JAKO200622234005936 to JAKO201505040785314
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   title     103069 non-null  string
 1   abstract  103070 non-null  string
 2   keywords  103070 non-null  string
 3   journal   103070 non-null  string
 4   type      103070 non-null  string
dtypes: string(5)
memory usage: 4.7 MB


In [19]:
df['journal'].nunique() # 저널 종류 수

268

In [6]:
df['journal'].value_counts() # 저널별 문서 수

한국콘텐츠학회논문지        7749
디지털융복합연구          5346
한국정보통신학회논문지       2864
대한토목학회논문집         2600
한국융합학회논문지         2512
                  ... 
대한한방부인과학회지          12
한국수소및신에너지학회논문집      11
한국항공운항학회지           11
한국해안해양공학회지          10
한국태양에너지학회 논문집       10
Name: journal, Length: 268, dtype: Int64

In [ ]:
# 최초 데이터의 document id - index 매핑
# 데이터 전처리 과정에서 데이터 제거 발생시, embedding 벡어테 대해서도 동일한 위치의 데이터를 삭제하기 위해 필요
old_id_index = pd.Series(range(len(df)), index=df.index)

---
# 초록(Abstract) SBERT Embedding
- 모든 초록 데이터를 SBERT Embedding해 놓기

In [5]:
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# 모델 다운로드 및 임베딩 테스트
sentences = ["테스트 할 것 입니다. 테스트 테스트 테스트."]

#model = SentenceTransformer('ys7yoo/sentence-roberta-large-kor-sts')
#model = SentenceTransformer("Huffon/sentence-klue-roberta-base")  #  https://huggingface.co/Huffon/sentence-klue-roberta-base
model = SentenceTransformer('smartmind/ko-sbert-augSTS-maxlength512') # https://huggingface.co/smartmind/ko-sbert-augSTS-maxlength512 # http://knlp.snu.ac.kr/index.html

In [6]:
df.shape

(103070, 6)

In [7]:
embed1 = []
for i in (tqdm(range(0,len(df.abstract)))):
    embed1.append(model.encode(df.abstract[i]))

 34%|███████████████████████▊                                              | 34996/103070 [5:12:30<6:46:05,  2.79it/s]

In [8]:
import numpy as np
# np.array(embed1)
np.save('./data/embed/df_complete_filtered.npy', np.array(embed1)) 

---
# 데이터 전처리
- 추가로 필요한 전처리 수행

## 결측값 처리

In [29]:
# title 결측값 제거 : <NA>
df.drop(index=df[df.isna().any(axis=1)].index, inplace=True)
df.isna().sum().sum()

0

In [30]:
# keywords 이상값 제거 : ';'
print(df.shape)
no_keywords_index = df[(df['keywords'] == '') | (df['keywords'] == ' ') | (df['keywords'] == ';')].index # keywords 값이 ';'인 데이터 2건
df.drop(index=no_keywords_index, inplace=True)
print(df.shape)

(103069, 5)
(103067, 5)


- Title 결측값 1건 제거
- Keywords 결측값 2건 제거
- => 총 3건 제거

## 제목(Title) 전처리

**전처리 방향**
- 일반 명사, 고유 명사, 외국어 추출
- 형태소 분석은 단순히 Mecab을 사용하는 것이 아닌, 아래의 예시와 같은 방향으로도 진행 됨
    - 예를 들어, 'e-commerce'의 경우, 'ecommerce'가 아닌 -> 'e' 'commerce'로 구분하여, 'e'와 'commerce' 2개의 다른 단어가 생기게 됨
    - 'e'도 엄밀히 말하면 의미가 있는 단어이며, 'e'의 포함 여부가 의미상 차이를 주기 때문에, 각각의 단어로 사전에 포함시키는 것으로 함
- length 1개짜리 text 유지
    - title이나 keyword에서 text length 1인 것들 있음
    - 한글의 경우 '혀', '술' 등 대부분 의미가 있기 때문에 별도 처리 없이 진행
    - 영어도 's'만 놓고 보면 별 의미 없어보이지만, 's 분석', 'a 교근', 'y 유전자' 이렇게 특정 주제에 대한 것으로 보여 그대로 사용함
- 숫자 유지
    - 숫자가 가지는 의미가 있기 때문에 유지
    - '119', '112', '2 형', '19 세기', '5 급', '3 차원', '10 종' 등
- 한자 유지
    - 한자가 쓰여진 경우 유사도 분석에서 더욱 명확할 것
    - '國內', '原電' 등
- Double (or more) space 제거
    - 전처리 과정에서 생긴 공백 제거
- 추출한 text 항목에서 중복되는 text도 유지

In [12]:
# title 처리
# title에서 한글 명사/영문 용어 추출

def extract_noun(sentence):
    # 한글(일반 명사, 고유 명사), 외국어, 숫자 추출
    nn_list = [re.sub(r'[^\w\s]','',text).strip().replace('   ', ' ').replace('  ', ' ').lower() for text, pos in mecab.pos(sentence) if pos in ('NNG','NNP','SL','SN')] # 특수문자 제거 
    #nn_list = list(dict.fromkeys(nn_list).keys()) # 중복 제거
    nn_res = ' '.join(nn_list).replace('   ', ' ').replace('  ', ' ') # 문자열 결합
    return nn_res

mecab = Mecab()
workers = os.cpu_count() * 2

df['title_nn'] = Parallel(n_jobs=workers)(delayed(extract_noun)(title) for title in df['title'])
df.head(3)

,title,abstract,keywords,journal,type,title_nn
JAKO200622234005936,AHP 기법을 활용한 CRM 평가요소의 상대적 중요도 분석,고객관계관리(CRM)의 전략적 중요성이 증대함에 따라 CRM에 대한 성과평가가 최근...,성과평가;고객관계관리;,CRM연구,JA,ahp 기법 활용 crm 평가 요소 상대 중요 분석
JAKO200622234005941,인터넷 오픈마켓 거래안전 요인과 소비자신뢰의 관계 연구,본 연구는 우리나라 전자상거래 업계에서 새로운 업태로 급격하게 성장하고 있는 인터넷...,인터넷오픈마켓;소비자신뢰;소비자만족;거래위험;,CRM연구,JA,인터넷 오픈 마켓 거래 안전 요인 소비자 신뢰 관계 연구
JAKO200622234005947,시스템 다이내믹스를 활용한 정보 기술 수용에 대한 동태적 모형 개발 - 휴대 전화 ...,기존의 초기 기술 수용 모델(TAM)과 후기 기술 수용 모델(PAM)을 사용한 연구...,시스템 다이내믹스;휴대 전화;,CRM연구,JA,시스템 다이내믹스 활용 정보 기술 수용 동태 모형 개발 휴대 전화 사용 중심


In [13]:
df.tail(3)

,title,abstract,keywords,journal,type,title_nn
JAKO201505040785309,낙동강 유역의 친환경 하천 준설을 위한 환경창 수립 방안,"국내의 하천 환경은 4대강 사업 이후로 많은 변화가 예상되며, 이 과정에서 하천관리...",하천준설;환경준설;환경창;준설시기;준설기간;,환경정책연구,JA,낙동강 유역 친환경 하천 준설 환경 창 수립 방안
JAKO201505040785312,쿠르노 경쟁하의 배출권 이월 및 차입과 감축기술개발투자,배출권 이월 및 차입은 감축기술개발투자에 영향을 미칠 수 있다. 배출권거래제하에서 ...,"쿠르노 경쟁모델;불완전경쟁;기술개발;배출권거래제;경제학문헌목록 주제분류 : L13,...",환경정책연구,JA,쿠르노 경쟁 배출 이월 차입 감축 기술 개발 투자
JAKO201505040785314,지리정보를 활용한 한국의 지질유산 정보화 구축 및 관리방안 제시,국내외으로 지질유산에 대한 관심과 선정 등이 지속적으로 이루어지고 있다. 그러나 지...,지질다양성;지질보전;지질관광;위치정보;세계자연유산;,환경정책연구,JA,지리 정보 활용 한국 지질 유산 정보 구축 관리 방안 제시


## 키워드(Keyword) 전처리

**전처리 방향**
- ; 또는 , 기준 분리
- 한글 명사, 외국어, 숫자 추출 (특수문자 제거)
- 한글과 영어가 붙어있는 단어 분리

In [17]:
# keywords 처리
# keywords ';' 기준 분리, 한글 명사/외국어(영어 등)/숫자 추출, (특수문자 제거로 인해 발생한) dobule space 제거

# ';' 또는 ',' 로 keywords 분리 (',' 로 구분 된 keywords 값도 있음)
def try_split(keywords):
    if keywords.find(';') == -1:
        kw_split = [x for x in keywords.split(',') if x != ''] # 마지막 element로 ''이 생성 될 수 있기 때문
        return kw_split
    else:
        kw_split = [x for x in keywords.split(';') if x != '']
        return kw_split

# 한 어절에서 영어와 한글을 분리하는 함수
# 숫자와 한글이 붙어있는 경우는 '2단계','3차원' 등의 그 자체로 중요한 용어
# 하지만 영어와 한글이 붙어있는 경우는  'f nucleatum속'처럼 분리를 해야 중요한 용어로 활용할 수 있을 것으로 보임 -> 'f nucleatum 속'
def separate_eng_kor(text_in_kw):
    splitted_word_segments = list()
    for each_word_seg in text_in_kw:
        kor_exist = False
        eng_exist = False
        temp_list = list()
        # 한 어절을 품사 태깅된 형태소로 분류하여 반환
        text_pos_list = mecab.pos(each_word_seg)
        # 한 어절의 분류된 형태소별 영어-한글 여부 확인
        for text, pos in text_pos_list:
            # 해당 형태소가 NNG 또는 NNP면 한글을 의미
            if pos in ('NNG','NNP'):
                kor_exist = True
                temp_list.append(text.strip().lower())
            # 해당 형태소가 SL이면 영어를 의미
            elif pos == 'SL':
                eng_exist = True
                temp_list.append(text.strip().lower())
        # 하나의 어절에 한글과 영어가 모두 있다면, 한글과 영어가 붙어있음을 의미하며, 사이를 띄워서 반환
        if kor_exist and eng_exist:
            result_text = ' '.join(temp_list)
        # 아니라면, 해당 어절 그대로 반환
        else:
            result_text = each_word_seg
        splitted_word_segments.append(result_text)
    return ' '.join(splitted_word_segments)

# 중요한 단어를 의미하는 'keyword'라는 특성을 고려하여, 하나의 text로 된 keyword가 쪼개지지 않도록, keyword 처리시엔 mecab 형태소 분석 X
# 단, 영어와 한글이 붙어있는 경우는 분리하고, 숫자와 한글/영어가 붙어있는 경우엔 그대로 유지
def extract_keyword(keywords):
    keyword_list = [separate_eng_kor(re.sub(r'[^\w\s]',' ',keyword).strip().replace('   ', ' ').replace('  ', ' ').split(' ')).lower() for keyword in try_split(keywords)] # 특수문자 제거
    #keyword_list = list(dict.fromkeys(keyword_list).keys()) # 처리한 키워드 내에서 중복 제거
    keyword_res = ' '.join(keyword_list).replace('   ', ' ').replace('  ', ' ') # 문자열 결합
    return keyword_res

mecab = Mecab()
workers = os.cpu_count() * 2

Parallel(n_jobs=workers)(delayed(extract_keyword)(keywords) for keywords in df['keywords'])

df['keyword'] = Parallel(n_jobs=workers)(delayed(extract_keyword)(keywords) for keywords in df['keywords'])
df.head(3)

,title,abstract,keywords,journal,type,title_nn,keyword
JAKO200622234005936,AHP 기법을 활용한 CRM 평가요소의 상대적 중요도 분석,고객관계관리(CRM)의 전략적 중요성이 증대함에 따라 CRM에 대한 성과평가가 최근...,성과평가;고객관계관리;,CRM연구,JA,ahp 기법 활용 crm 평가 요소 상대 중요 분석,성과평가 고객관계관리
JAKO200622234005941,인터넷 오픈마켓 거래안전 요인과 소비자신뢰의 관계 연구,본 연구는 우리나라 전자상거래 업계에서 새로운 업태로 급격하게 성장하고 있는 인터넷...,인터넷오픈마켓;소비자신뢰;소비자만족;거래위험;,CRM연구,JA,인터넷 오픈 마켓 거래 안전 요인 소비자 신뢰 관계 연구,인터넷오픈마켓 소비자신뢰 소비자만족 거래위험
JAKO200622234005947,시스템 다이내믹스를 활용한 정보 기술 수용에 대한 동태적 모형 개발 - 휴대 전화 ...,기존의 초기 기술 수용 모델(TAM)과 후기 기술 수용 모델(PAM)을 사용한 연구...,시스템 다이내믹스;휴대 전화;,CRM연구,JA,시스템 다이내믹스 활용 정보 기술 수용 동태 모형 개발 휴대 전화 사용 중심,시스템 다이내믹스 휴대 전화


In [18]:
df.tail(3)

,title,abstract,keywords,journal,type,title_nn,keyword
JAKO201505040785309,낙동강 유역의 친환경 하천 준설을 위한 환경창 수립 방안,"국내의 하천 환경은 4대강 사업 이후로 많은 변화가 예상되며, 이 과정에서 하천관리...",하천준설;환경준설;환경창;준설시기;준설기간;,환경정책연구,JA,낙동강 유역 친환경 하천 준설 환경 창 수립 방안,하천준설 환경준설 환경창 준설시기 준설기간
JAKO201505040785312,쿠르노 경쟁하의 배출권 이월 및 차입과 감축기술개발투자,배출권 이월 및 차입은 감축기술개발투자에 영향을 미칠 수 있다. 배출권거래제하에서 ...,"쿠르노 경쟁모델;불완전경쟁;기술개발;배출권거래제;경제학문헌목록 주제분류 : L13,...",환경정책연구,JA,쿠르노 경쟁 배출 이월 차입 감축 기술 개발 투자,쿠르노 경쟁모델 불완전경쟁 기술개발 배출권거래제 경제학문헌목록 주제분류 l13 l5...
JAKO201505040785314,지리정보를 활용한 한국의 지질유산 정보화 구축 및 관리방안 제시,국내외으로 지질유산에 대한 관심과 선정 등이 지속적으로 이루어지고 있다. 그러나 지...,지질다양성;지질보전;지질관광;위치정보;세계자연유산;,환경정책연구,JA,지리 정보 활용 한국 지질 유산 정보 구축 관리 방안 제시,지질다양성 지질보전 지질관광 위치정보 세계자연유산


In [ ]:
# 전처리한 데이터(dataframe) 저장
# df.to_csv('./data/df_prep.csv')

---
# Train/Test 분리
1) 전처리 완료된 데이터셋 train / test 분리 
    - train:test = 9:1
    - random seed = 777
    - stratify by = 'journal'
2) SBERT 임베딩 벡터 train / test 분리

In [ ]:
# train/test 분리

from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1, random_state=777, stratify=df['journal']) # journal 기준 stratify
print(train.shape)
print(test.shape)

(92763, 5)
(10307, 5)


In [ ]:
# train, test 데이터 csv 저장
# train.to_csv('./data/train.csv')
# test.to_csv('./data/test.csv')

In [ ]:
# SBERT 임베딩 벡터 train/test 분리

# 위 기존 데이터의 document id - index 매핑된 정보에서 현재 분석에서 사용하는 데이터의 document id - index만 필터링
id_index_kept = old_id_index[df.index]

# 현재 사용중인 (전처리 완료한) 데이터에서 Document id - Index 매핑
all_id_index = pd.Series(range(len(df)), index=df.index)

# 전체 데이터셋 기준 train, test 데이터 index 위치 저장 : document id - index 매핑
## train, test 데이터별 SBERT embedding vectors 추출용 (SBERT Embedding은 데이터 전체로 되어 있기 때문)
train_id_index_whole = all_id_index[train.index] # (doc_id, index)[doc_id]
test_id_index_whole = all_id_index[test.index]

# 데이터 전처리 과정에서 제거한 데이터/index를 SBERT embedding matrix에서도 제거
# 이를 위해, 전처리간 삭제한 데이터의 document id -- index를 뺀 정보를 저장한 변수를 활용
embed = embed[id_index_kept.values] # 삭제한 데이터의 index에 해당하는 데이터 빼고 임베딩 matrix 저장

# train data의 embedding vectors 추출
# train 데이터의 document id에 해당하는 index를 사용해 동일한 index 위치의 embedding vectors만 필터링
train_embed = embed[train_id_index_whole.values]

# test data의 embedding vectors 추출
# test 데이터의 document id에 해당하는 index를 사용해 동일한 index 위치의 embedding vectors만 필터링
test_embed = embed[test_id_index_whole.values]

In [208]:
# train embedding numpy 배열 파일로 저장
# np.save('./data/train_embed', train_embed)
# np.save('./data/test_embed', test_embed)

# train embedding npy 파일 다시 읽기
# train_embed = np.load('./data/train_embed.npy')
# test_embed = np.load('./data/test_embed.npy')

---
# Train 데이터에서 필요한 변수 선언
- **이후 모델링 과정에서 불러와서 사용**

In [3]:
# 전처리한 train 데이터(dataframe) 다시 읽어오기
# train = pd.read_csv('./data/train.csv', index_col=0)
print(train.shape)

# Train y 
train_y = train['journal'] # Train 데이터 doc id - journal
print(train_y.shape)

# train doc id - index : Train 데이터셋 기준 각 document id - index 매핑 (데이터셋에서의 위치)
train_id_index = pd.Series(range(len(train.index)), index=train.index)
print(len(train_id_index))

# Train SBERT embedding
# train_embed = np.load('./data/train_embed.npy')
print(train_embed.shape)

(92760, 7)
(92760,)
92760
(92760, 768)


In [92]:
# 저널명 - Index 매핑  (저널 고정 순서)
journ_order = train.groupby('journal').groups.keys()
journ_index = pd.Series(range(len(journ_order)), index=journ_order)

print(journ_index[:5])
print(journ_index[-5:])

journal
CRM연구                                                      0
Child Health Nursing Research                              1
Clinical and Experimental Reproductive Medicine            2
Clinics in Shoulder and Elbow                              3
Communications for Statistical Applications and Methods    4
dtype: int64
journal
해양환경안전학회지    263
혜화의학회지       264
화약ㆍ발파        265
환경영향평가       266
환경정책연구       267
dtype: int64


# Train 데이터의 JTM & JKM 생성
- 저널별 Title 텍스트 매트릭스 생성
- 저널별 Keyword 텍스트 매트릭스 생성

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# Journal-Title Matrix (JTM)
# journal별 title CountVectorize
train_journ_tt = train.groupby('journal')['title_nn'].apply(' '.join) # journal별로 모든 논문들 title text 합치기
count_vect_tt = CountVectorizer(min_df=1, ngram_range=(1,1)) # unigram
jtm = count_vect_tt.fit_transform(train_journ_tt) # sparse matrix
print(jtm.shape)

# csr matrix -> dataframe
# jtm_df = pd.DataFrame(jtm.toarray(), index=train_journ_tt.index, columns=count_vect_tt.get_feature_names_out())

# ================================================= #
# Journal-Keyword Matrix (JKM)
# journal별 keyword CountVectorize
train_journ_kw = train.groupby('journal')['keyword'].apply(' '.join) # journal별로 모든 논문들 keyword text 합치기
count_vect_kw = CountVectorizer(min_df=1, ngram_range=(1,1)) # unigram
jkm = count_vect_kw.fit_transform(train_journ_kw) # sparse matrix
print(jkm.shape)

# csr matrix -> dataframe
# jkm_df = pd.DataFrame(jkm.toarray(), index=train_journ_kw.index, columns=count_vect_kw.get_feature_names_out())

(268, 43041)
(268, 135402)


---
# Test 데이터에서 필요한 변수 선언
- **이후 모델링 과정에서 불러와서 사용**

In [264]:
# 전처리한 test 데이터(dataframe) 다시 읽어오기
# test = pd.read_csv('./data/test.csv', index_col=0)
print(test.shape)

# Test y
test_y = test['journal'] # Test 데이터 document id - journal
print(test_y.shape)

# Test 데이터셋 기준 document id - index 매핑 (데이터셋에서의 위치)
test_id_index = pd.Series(range(len(test.index)), index=test.index)
print(len(test_id_index))

# Test SBERT embedding npy 파일 다시 읽기
# test_embed = np.load('./data/test_embed.npy')
print(test_embed.shape)

(10307, 7)
(10307,)
10307
(10307, 768)
